# 1

In [1]:
import pickle

with open('./var_02/task_1_item.pkl', "rb") as f:
    data = pickle.load(f)

In [2]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["database"]

collection = db.sample_collection
collection.drop()
_ = collection.insert_many(data)

In [3]:
import json

with open("1_sorted_by_salary.json", "w", encoding="utf-8") as outfile: 
    json.dump([{key: str(e[key]) if type(e[key]) not in ['int', 'float'] else e[key] for key in e} 
               for e in collection.find({}, limit=10).sort({'salary':-1})], outfile, ensure_ascii=True)
    
with open("1_filtered_by_age_sorted_by_salary.json", "w", encoding="utf-8") as outfile: 
    json.dump([{key: str(e[key]) if type(e[key]) not in ['int', 'float'] else e[key] for key in e} 
               for e in collection.find({'age': {'$lt': 30}}, limit=15).sort({'salary':-1})], outfile, ensure_ascii=True)
    
with open("1_filtered_by_complex_sorted_by_age.json", "w", encoding="utf-8") as outfile: 
    json.dump([{key: str(e[key]) if type(e[key]) not in ['int', 'float'] else e[key] for key in e} 
               for e in collection.find({'city': 'Варшава', 'job': {'$in': ['Программист', 'Инженер', 'IT-специалист']}}, 
                                              limit=10).sort({'age':1})], 
              outfile, ensure_ascii=True)
    
with open("1_count.json", "w", encoding="utf-8") as outfile: 
    json.dump(collection.count_documents({
        "age": {"$gt": 40, "$lt": 60},
        "year": {"$gt": 2018, "$lt": 2023},
        "$or":  [{"salary": {"$gt": 50000, "$lt": 75000}}, 
                 {"salary": {"$gt": 125000, "$lt": 150000}}]
    }), 
              outfile, ensure_ascii=True)

# 2

In [4]:
import pandas as pd

def parse_df(df):
    data = df.to_dict()
    keys = list(data.keys())
    n = len(data[keys[0]])
    return [{key : data[key][i] for key in keys} for i in range(n)]

data = pd.read_csv('./var_02/task_2_item.csv', sep=';')
data = parse_df(data)
_ = collection.insert_many(data)

In [5]:
with open("2_stats.json", "w", encoding="utf-8") as outfile: 
    json.dump([e for e in collection.aggregate([
        {"$group": {
            "_id":  "salary",
            "min": {"$min": "$salary"},
            "max": {"$max": "$salary"},
            "avg": {"$avg": "$salary"}
    }}])], outfile, ensure_ascii=True)

with open("2_freq.json", "w", encoding="utf-8") as outfile: 
    json.dump([e for e in collection.aggregate([
        {"$group": {"_id": "$job", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}}
    ])], outfile, ensure_ascii=True)

with open("2_salary_stats_city.json", "w", encoding="utf-8") as outfile: 
    json.dump([e for e in collection.aggregate([
        {"$group": {
            "_id":  "$city",
            "min": {"$min": "$salary"},
            "max": {"$max": "$salary"},
            "avg": {"$avg": "$salary"}
    }}])], outfile, ensure_ascii=True)
    
with open("2_salary_stats_job.json", "w", encoding="utf-8") as outfile: 
    json.dump([e for e in collection.aggregate([
        {"$group": {
            "_id":  "$job",
            "min": {"$min": "$salary"},
            "max": {"$max": "$salary"},
            "avg": {"$avg": "$salary"}
    }}])], outfile, ensure_ascii=True)

with open("2_age_stats_city.json", "w", encoding="utf-8") as outfile: 
    json.dump([e for e in collection.aggregate([
        {"$group": {
            "_id":  "$city",
            "min": {"$min": "$age"},
            "max": {"$max": "$age"},
            "avg": {"$avg": "$age"}
    }}])], outfile, ensure_ascii=True)
    
with open("2_age_stats_job.json", "w", encoding="utf-8") as outfile: 
    json.dump([e for e in collection.aggregate([
        {"$group": {
            "_id":  "$job",
            "min": {"$min": "$age"},
            "max": {"$max": "$age"},
            "avg": {"$avg": "$age"}
    }}])], outfile, ensure_ascii=True)
    
with open("2_min_age_max_salary.json", "w", encoding="utf-8") as outfile: 
    json.dump([e for e in collection.aggregate([
        {"$group": {
            "_id":  "$age",
            "max": {"$max": "$salary"}
    }},
        {"$group": {
            "_id":  "result",
            "age": {"$min": "$_id"},
            "max": {"$max": "$max"}
    }}])], outfile, ensure_ascii=True)
    
with open("2_age_stats_city_with_conditions.json", "w", encoding="utf-8") as outfile: 
    json.dump([e for e in collection.aggregate([
        {"$match": {"salary": {"$gt": 50000}}},
        {"$group": {
            "_id":  "$city",
            "min": {"$min": "$age"},
            "max": {"$max": "$age"},
            "avg": {"$avg": "$age"}
        }},
        {"$sort": {"_id": 1}}
    ])], outfile, ensure_ascii=True)

with open("2_salary_stats_city_with_conditions.json", "w", encoding="utf-8") as outfile: 
    json.dump([e for e in collection.aggregate([
        {"$match": {
            "city": {"$in": ["Авилес", "Аликанте", "Алма-Ата"]},
            "job": {"$in": ['Программист', 'Инженер', 'IT-специалист']},
            "$or": [{"age": {"$gt": 18, "$lt": 25}}, 
                    {"age": {"$gt": 50, "$lt": 65}}]
        }},
        {"$group": {
            "_id":  "$city",
            "min": {"$min": "$salary"},
            "max": {"$max": "$salary"},
            "avg": {"$avg": "$salary"}
        }}
    ])], outfile, ensure_ascii=True)
    
with open("2_custom.json", "w", encoding="utf-8") as outfile: 
    json.dump([e for e in collection.aggregate([
        {"$match": {"salary": {"$gt": 50000}}},
        {"$group": {
            "_id":  "$year",
            "min": {"$min": "$age"},
            "max": {"$max": "$age"},
            "avg": {"$avg": "$age"}
        }},
        {"$sort": {"_id": 1}}
    ])], outfile, ensure_ascii=True)

# 3

In [6]:
import msgpack
    
with open("./var_02/task_3_item.msgpack", "rb") as data_file:
    byte_data = data_file.read()
    data = msgpack.unpackb(byte_data)
    
_ = collection.insert_many(data)

In [7]:
collection.delete_many({
    "$or": [{"salary": {"$lt": 25000}},
            {"salary": {"$gt": 175000}}]})

DeleteResult({'n': 366, 'ok': 1.0}, acknowledged=True)

In [8]:
collection.update_many({}, {"$inc": {"age" : 1}})

UpdateResult({'n': 1718, 'nModified': 1718, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [9]:
collection.update_many({"job": {"$in" : ['Программист', 'Инженер', 'IT-специалист']}}, 
                       {"$mul": {"salary" : 1.05}})

UpdateResult({'n': 310, 'nModified': 310, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [10]:
collection.update_many({"city": {"$in" : ["Авилес", "Аликанте", "Алма-Ата"]}}, 
                       {"$mul": {"salary" : 1.07}})

UpdateResult({'n': 67, 'nModified': 67, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [11]:
collection.update_many({
    "city": "Авилес",
    "age": {"$gt": 40},
    "job": {"$in" : ['Программист', 'IT-специалист']}
}, 
                       {"$mul": {"salary" : 1.1}})

UpdateResult({'n': 3, 'nModified': 3, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [12]:
collection.delete_many({
    "$and": [{"job": 'Инженер'},
            {"age": {"$gt": 40}}]})

DeleteResult({'n': 64, 'ok': 1.0}, acknowledged=True)

# 4

In [13]:
collection = db.custom_collection
collection.drop()

In [14]:
with open('data1.json', encoding='utf-8') as f:
    data1 = json.load(f)

_ = collection.insert_many(data1)    

In [15]:
def parse_df(df):
    data = df.to_dict()
    keys = list(data.keys())
    n = len(data[keys[0]])
    return [{key : data[key][i] for key in keys} for i in range(n)]

data2 = pd.read_csv('data2.csv')
data2 = parse_df(data2)
_ = collection.insert_many(data2)    

In [16]:
# выборка 
with open("3_sorted_by_rating.json", "w", encoding="utf-8") as outfile: 
    json.dump([{key: str(e[key]) if type(e[key]) not in ['int', 'float'] else e[key] for key in e} 
               for e in collection.find({}, limit=10).sort({'rating':-1})], outfile, ensure_ascii=True)

with open("3_sorted_by_user_id.json", "w", encoding="utf-8") as outfile: 
    json.dump([{key: str(e[key]) if type(e[key]) not in ['int', 'float'] else e[key] for key in e} 
               for e in collection.find({}, limit=10).sort({'user_id':-1})], outfile, ensure_ascii=True)
    
with open("3_filtered_by_ws_sorted_by_rating.json", "w", encoding="utf-8") as outfile: 
    json.dump([{key: str(e[key]) if type(e[key]) not in ['int', 'float'] else e[key] for key in e} 
               for e in collection.find({'watching_status': 0}, limit=15).sort({'rating':-1})], outfile, ensure_ascii=True)
    
with open("3_filtered_by_complex_sorted_by_rating.json", "w", encoding="utf-8") as outfile: 
    json.dump([{key: str(e[key]) if type(e[key]) not in ['int', 'float'] else e[key] for key in e} 
               for e in collection.find({'anime_id': 138, 'watching_status': {'$in': ['0', '1']}}, 
                                              limit=10).sort({'rating':1})], 
              outfile, ensure_ascii=True)
    
with open("3_count.json", "w", encoding="utf-8") as outfile: 
    json.dump(collection.count_documents({
        "user_id" : 0
    }), 
              outfile, ensure_ascii=True)

In [17]:
# выбора с агрегацией
with open("3_stats.json", "w", encoding="utf-8") as outfile: 
    json.dump([e for e in collection.aggregate([
        {"$group": {
            "_id":  "rating",
            "min": {"$min": "$rating"},
            "max": {"$max": "$rating"},
            "avg": {"$avg": "$rating"}
    }}])], outfile, ensure_ascii=True)

with open("3_freq.json", "w", encoding="utf-8") as outfile: 
    json.dump([e for e in collection.aggregate([
        {"$group": {"_id": "$watching_status", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}}
    ])], outfile, ensure_ascii=True)

with open("3_rating_stats_anime.json", "w", encoding="utf-8") as outfile: 
    json.dump([e for e in collection.aggregate([
        {"$group": {
            "_id":  "$anime_id",
            "min": {"$min": "$rating"},
            "max": {"$max": "$rating"},
            "avg": {"$avg": "$rating"}
    }}])], outfile, ensure_ascii=True)
    
with open("3_user_stats_episodes.json", "w", encoding="utf-8") as outfile: 
    json.dump([e for e in collection.aggregate([
        {"$group": {
            "_id":  "$user_id",
            "sum": {"$sum": "$watched_episodes"},
    }}])], outfile, ensure_ascii=True)


with open("3_user_stats_rating_with_conditions.json", "w", encoding="utf-8") as outfile: 
    json.dump([e for e in collection.aggregate([
        {"$match": {"watching_status": {"$gt": 0}}},
        {"$group": {
            "_id":  "$user_id",
            "min": {"$min": "$rating"},
            "max": {"$max": "$rating"},
            "avg": {"$avg": "$rating"}
        }},
        {"$sort": {"_id": 1}}
    ])], outfile, ensure_ascii=True)

In [18]:
# обновление/удаление данных

collection.delete_many({"watching_status": 2})

DeleteResult({'n': 206460, 'ok': 1.0}, acknowledged=True)

In [19]:
collection.update_many({}, {"$inc": {"user_id" : 1}})

UpdateResult({'n': 139917, 'nModified': 139917, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [20]:
collection.update_many({"rating": {"$gt": 8}}, 
                       {"$inc": {"watching_status" : 10}})

UpdateResult({'n': 2454, 'nModified': 2454, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [21]:
collection.update_many({"anime_id": 100}, 
                       {"$mul": {"watching_status" : 0}})

UpdateResult({'n': 28, 'nModified': 28, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [22]:
collection.delete_many({"anime_id": {"$gt": 500}})

DeleteResult({'n': 125467, 'ok': 1.0}, acknowledged=True)